In [2]:
import os
from langchain.llms import HuggingFaceHub
from langchain.embeddings import OpenAIEmbeddings, HuggingFaceInstructEmbeddings
os.environ['HUGGINGFACEHUB_API_TOKEN'] = # your unique API Token

In [3]:
from langchain.document_loaders import PyPDFLoader

directory='report.pdf' # upload the desired pdf file

def load_docs(directory):
    loader = PyPDFLoader(directory)
    documents= loader.load_and_split()
    return documents

documents= load_docs(directory)
len(documents)

20

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def split_docs(documents,chunk_size=1000,chunk_overlap=20):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    docs= text_splitter.split_documents(documents)
    return docs

docs= split_docs(documents)
print(len(docs))

31


In [5]:
print(docs[25].page_content)

CHAPTER 3
RESULT AND DISCUSSION
3.1 Resut
As explained in the section 2.2, we used different approaches to test our data for accurate
prediction. The accuracy thus presented is:
Classifier Training Accuracy Testing Accuracy
TD-if + Naive Bayes 28.72744539411206% 25.783475783475783%
Passive Aggressive Classifier 99.82190560997329% 69.7508896797153%
Classifier Training Accuracy Testing Accuracy
Bigram and Tigram
TD-if + Naive Bayes 27.326685660018995% 24.501424501424502%
Bigram and Tigram
Passive Aggressive Classifier 99.84416740872662% 67.88256227758008%
3.2 Discussion
We found that prediction based on Naive Bayes Algorithm gives poor accuracy compared
to that of Passive Aggressive Classifier. This is because the ”naive” assumption is that
features are independent with each other which may not be true in practice and limits its
efficiency. In disease prediction, depending on the disease, symptoms may depend on each


In [6]:
print(docs[20].page_content)

If the decision is correct, there is no change in weight. However, it adjusts the weight
to better classify future instances if its reading comes to be wrong. Passive Aggressive
Classifier (PAC) works by taking a set of data and dividing it into two groups:
1.Training Set: PAC learns how to correctly classify a data in the set.
2.Testing Set: Once learned, PAC tests the data and the accuracy is calculated on
the basis of that test.
6


In [7]:
embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl")
text= "Hello World"
query_result = embeddings.embed_query(text)
len(query_result)

C:\Users\ayush\anaconda3\lib\site-packages\InstructorEmbedding\instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


load INSTRUCTOR_Transformer
max_seq_length  512


768

In [8]:
import pinecone
from langchain.vectorstores import Pinecone
pinecone.init(
    api_key= #enter your own unique api key
    environment=#enter your own unique enviroment name
)

index_name = #"project"

index = Pinecone.from_documents(docs, embeddings, index_name=index_name)

In [9]:
def get_similar_docs(query, k=2, score=False):
  if score:
    similar_docs = index.similarity_search_with_score(query, k=k)
  else:
    similar_docs = index.similarity_search(query, k=k)
  return similar_docs

In [10]:
import openai
from langchain.chains.question_answering import load_qa_chain

llm = HuggingFaceHub(repo_id="google/flan-t5-xxl", model_kwargs={"temperature":0.5,"max_length":510450})
chain = load_qa_chain(llm, chain_type="stuff")

def get_answer (query):
  similar_docs = get_similar_docs(query)
  answer = chain.run(input_documents=similar_docs, question=query)
  return answer

C:\Users\ayush\anaconda3\lib\site-packages\huggingface_hub\utils\_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


In [11]:
query = "what is formula for tf-idf" # Questions from within the document
get_answer (query)

'TF-IDF( t, d, D ) = TF( t, d)IDF(t, D) 5'

In [12]:
query = "What are the ethical effects of ML?"
get_answer(query)

'The prediction may not be accurate with the real world scenario. 2. Actual human pain/sentiment cannot be calculated to make better predictions. 3. Only the surface potential of NLP is used. 4. The data collected was not sufficient and does not have proportional distribution for large scale prediction.'

In [14]:
query = "How far is the sun" # Question outside the document
get_answer(query) # Output : (I don't know)

"I don't know"